# Descrizione notebook

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
import math
import random
from itertools import combinations
from ipynb.fs.defs.creazione_CSV import normalization
from ipynb.fs.defs.UBCF import pearson_correlation_coefficient, score_user_item_ubcf, recommendation_ubcf
from ipynb.fs.defs.IBCF import cosine_similarity, score_user_item_ibcf, recommendation_ibcf

In [2]:
pd.set_option('display.max_columns', 102)
# lettura della matrice con gli utenti che hanno valutato tutte le barzellette
complete_ratings = pd.read_csv('./data/complete_ratings_matrix.csv')

complete_ratings = complete_ratings.iloc[:50, :]

# lettura dei rating non normalizzati dal relativo CSV (serve solo per calcolare la predizione, nella
# quale serve calcolare il rating medio dell'utente)
no_normalized_ratings_df = pd.read_csv('./data/no_normalized_ratings.csv')
complete_ratings

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,2,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,7.82,-3.83,8.64,8.98,-4.32,-3.20,-4.90,-0.92,-1.46,3.64,3.16,9.03,0.97,-1.31,-6.50,-3.20,8.64,-2.14,0.10,9.03,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,6,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,-7.04,-8.83,-7.91,1.07,-1.89,-6.36,-4.22,-9.27,-9.81,-7.82,-0.63,-6.31,-7.48,-6.07,-3.45,-3.20,0.53,-9.27,-9.56,-5.49,-1.12,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
2,8,100,6.84,3.16,9.17,-6.21,-8.16,-1.70,9.27,1.41,-5.19,-4.42,8.20,-7.86,-6.94,-7.96,0.29,-9.90,-7.09,-7.18,1.02,-0.29,-4.71,-7.43,1.12,-8.25,3.79,0.10,3.45,-3.45,7.57,-4.17,-0.44,9.27,5.83,-3.40,-5.44,1.60,-7.14,3.01,-6.07,5.68,-6.46,-4.42,-8.98,0.53,-1.26,-4.42,-8.93,8.93,-0.10,7.43,-4.51,-7.38,4.17,-9.03,-2.14,1.31,-5.68,-9.08,-6.21,2.48,-3.20,-6.12,-2.91,-9.22,-2.62,-7.33,-3.25,-4.22,-9.81,-2.09,-9.90,-3.79,0.49,-7.86,-9.85,1.80,-4.17,-2.38,-9.13,-8.88,3.54,-4.56,-5.44,-5.97,-9.90,-0.34,-4.13,-0.10,-0.24,-7.96,7.23,-1.12,-0.10,-5.68,-3.16,-3.35,2.14,-0.05,1.31,0.00
3,9,100,-3.79,-3.54,-9.42,-6.89,-8.74,-0.29,-5.29,-8.93,-7.86,-1.60,-2.91,-0.29,-4.85,-0.49,-8.74,-6.99,-8.74,-2.91,-3.35,-0.29,3.98,-1.60,-0.29,1.21,3.79,2.91,4.85,2.28,0.97,-0.29,6.36,0.53,-0.29,4.66,3.35,0.97,-3.11,-1.84,4.42,-4.42,-1.41,1.41,-5.34,-6.99,1.41,5.10,1.84,-0.29,0.53,-0.29,-6.99,-3.54,6.36,1.84,3.35,-0.29,-0.29,-5.29,-6.89,-5.29,1.21,5.53,1.41,-6.50,4.85,4.85,2.62,1.41,6.36,1.41,-4.76,4.17,-2.04,-6.31,-0.29,-0.29,0.78,-0.29,-0.29,-0.29,-0.29,-3.20,-0.29,-3.20,-3.40,-0.29,-0.29,5.73,4.56,-2.23,4.37,-0.29,4.17,-0.29,-0.29,-0.29,-0.29,-0.29,-3.40,-4.95
4,12,100,1.31,1.80,2.57,-2.38,0.73,0.73,-0.97,5.00,-7.23,-1.36,3.83,1.75,5.63,-2.86,-1.80,-2.04,5.53,-0.29,-0.58,1.36,2.43,0.63,-4.61,-4.37,-1.65,-1.36,7.57,-0.15,5.97,-3.50,0.24,4.27,-0.68,-3.79,7.62,5.15,2.43,0.10,0.83,-2.04,0.53,2.33,-2.28,-5.15,-0.58,-1.26,2.04,-1.41,2.38,6.60,2.04,-1.99,-2.77,-0.49,1.75,0.92,-6.89,-5.29,0.68,-2.38,-0.15,-0.24,1.46,-2.14,2.43,-2.43,-5.10,-0.49,0.15,-2.62,-2.91,1.12,1.70,0.00,2.04,3.06,-4.47,2.04,1.12,3.30,1.46,0.68,-2.57,-0.63,0.58,-0.63,1.89,2.38,6.21,-2.52,1.46,1.70,0.29,-3.30,3.45,5.44,4.08,2.48,4.51,4.66
5,14,100,9.22,9.27,9.22,8.30,7.43,0.44,3.50,8.16,5.97,8.98,3.74,5.87,8.69,6.31,1.07,-9.13,3.69,-7.33,3.88,-7.48,8.45,9.17,9.22,9.03,9.27,4.42,8.69,1.75,7.57,-7.14,8.93,6.80,0.87,7.23,8.79,8.79,7.43,8.59,8.54,9.17,8.20,8.54,8.20,9.17,9.08,2.86,-2.38,7.43,1.17,-0.29,5.00,8.35,7.62,8.16,8.06,8.35,9.22,-9.27,7.14,9.08,4.08,-9.81,-7.14,

# Preparazione Training set e Test set

In [3]:
test_set = []
no_normalized_train_set = complete_ratings.copy(deep=True)  
# per ogni riga (<indice_riga, riga>) e per ogni barzelletta della riga si salva la
# coppia <nome_barzelletta, rating> in una lista (altrimenti si perde il riferimento al nome della barzelletta)
# poi si prendono n coppie random, si salvano le informazioni nel test set, e si toglie dalla matrice completa
# quello che si è preso almeno si ottiene il training set di conseguenza
for index, row in no_normalized_train_set.iterrows():
    lst_app = []
    for i, elem in row[2:].items():
        lst_app.append((i, elem))
    lst_pair = random.sample(lst_app, 20)
    # pair = random.choice(lst_app)
    for pair in lst_pair:
        app = {}
        app["user_id"] = int(row["user_id"])
        app[pair[0]] = pair[1]
        test_set.append(app)
        no_normalized_train_set.loc[no_normalized_train_set["user_id"] == int(row["user_id"]), pair[0]] = np.nan
    
# test_set_df = pd.DataFrame(test_set)

In [4]:
no_normalized_train_set

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,2,100,4.08,-0.29,NaN,NaN,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,NaN,NaN,4.66,-0.68,NaN,-1.21,0.87,NaN,8.35,9.17,0.05,NaN,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,NaN,-1.80,0.73,7.09,NaN,-0.87,7.91,7.82,-3.83,NaN,8.98,-4.32,-3.20,-4.90,NaN,-1.46,3.64,3.16,NaN,0.97,-1.31,NaN,-3.20,8.64,-2.14,NaN,9.03,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,NaN,4.85,4.85,NaN,8.98,4.51,-0.05,3.69,4.56,NaN,NaN,-4.95,-0.29,NaN,NaN,-2.14,3.06,0.34,-4.32,1.07
1,6,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,NaN,-6.65,-1.80,NaN,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,NaN,-7.77,-2.23,NaN,-5.24,NaN,-0.68,-6.94,-3.93,NaN,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,NaN,-7.04,-8.83,-7.91,1.07,-1.89,-6.36,NaN,NaN,NaN,-7.82,-0.63,-6.31,-7.48,NaN,-3.45,-3.20,0.53,-9.27,-9.56,NaN,-1.12,NaN,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,NaN,NaN,-6.21,-2.86,-7.62,-6.12,-0.29,NaN,-3.93,-1.94,-5.68,NaN,NaN,-6.89,NaN,-2.96,-2.18,-3.35,0.05,NaN,-5.05,-3.45
2,8,100,6.84,3.16,9.17,-6.21,-8.16,NaN,9.27,NaN,-5.19,-4.42,8.20,-7.86,-6.94,NaN,0.29,NaN,-7.09,-7.18,1.02,NaN,NaN,NaN,1.12,-8.25,3.79,NaN,3.45,-3.45,NaN,-4.17,-0.44,NaN,5.83,NaN,-5.44,NaN,-7.14,3.01,-6.07,NaN,-6.46,-4.42,NaN,0.53,-1.26,-4.42,-8.93,8.93,-0.10,7.43,-4.51,-7.38,4.17,-9.03,-2.14,1.31,-5.68,NaN,-6.21,NaN,-3.20,-6.12,-2.91,-9.22,-2.62,-7.33,-3.25,-4.22,-9.81,-2.09,-9.90,-3.79,0.49,NaN,-9.85,1.80,-4.17,NaN,-9.13,-8.88,3.54,-4.56,NaN,-5.97,-9.90,-0.34,-4.13,-0.10,-0.24,-7.96,7.23,-1.12,-0.10,-5.68,-3.16,-3.35,2.14,-0.05,NaN,0.00
3,9,100,-3.79,NaN,-9.42,-6.89,-8.74,NaN,-5.29,-8.93,NaN,NaN,-2.91,-0.29,-4.85,NaN,-8.74,-6.99,-8.74,-2.91,NaN,NaN,3.98,NaN,-0.29,1.21,3.79,2.91,4.85,2.28,0.97,-0.29,6.36,0.53,NaN,4.66,3.35,0.97,-3.11,NaN,NaN,-4.42,-1.41,1.41,-5.34,-6.99,1.41,5.10,1.84,-0.29,0.53,-0.29,-6.99,-3.54,NaN,1.84,3.35,-0.29,-0.29,-5.29,-6.89,-5.29,NaN,5.53,1.41,-6.50,4.85,4.85,2.62,1.41,6.36,1.41,-4.76,4.17,NaN,NaN,-0.29,-0.29,0.78,-0.29,-0.29,-0.29,-0.29,-3.20,-0.29,-3.20,-3.40,-0.29,-0.29,5.73,4.56,-2.23,4.37,NaN,NaN,-0.29,NaN,-0.29,NaN,NaN,-3.40,-4.95
4,12,100,1.31,1.80,2.57,-2.38,0.73,NaN,-0.97,5.00,-7.23,-1.36,3.83,1.75,5.63,-2.86,-1.80,NaN,5.53,NaN,-0.58,1.36,2.43,0.63,NaN,-4.37,-1.65,-1.36,7.57,-0.15,5.97,NaN,0.24,4.27,-0.68,-3.79,NaN,5.15,NaN,NaN,0.83,-2.04,0.53,NaN,-2.28,NaN,-0.58,-1.26,2.04,-1.41,2.38,6.60,2.04,-1.99,-2.77,-0.49,1.75,0.92,-6.89,-5.29,NaN,-2.38,-0.15,NaN,1.46,-2.14,2.43,-2.43,-5.10,NaN,0.15,NaN,-2.91,1.12,1.70,0.00,NaN,3.06,NaN,2.04,NaN,3.30,1.46,0.68,NaN,-0.63,0.58,NaN,1.89,NaN,6.21,-2.52,1.46,1.70,0.29,-3.30,3.45,5.44,4.08,2.48,4.51,4.66
5,14,100,9.22,NaN,9.22,8.30,NaN,0.44,3.50,8.16,NaN,8.98,3.74,5.87,8.69,NaN,1.07,NaN,3.69,-7.33,3.88,NaN,8.45,9.17,9.22,9.03,9.27,4.42,NaN,1.75,7.57,-7.14,8.93,NaN,0.87,7.23,8.79,8.79,7.43,8.59,8.54,9.17,NaN,NaN,8.20,9.17,9.08,2.86,-2.38,7.43,1.17,-0.29,5.00,8.35,7.62,8.16,8.06,8.35,NaN,-9.27,7.14,9.08,4.08,-9.81,NaN,9.32,NaN,7.57,-5.78,-6.55,-7.23,NaN,-7.57,8.93,NaN,NaN,-6.21,8.35,NaN,8.69,-8.45,8.79,8.88,-8.69,7.77,-0.83,7.62,8.20,5.44,8.93,8.45,NaN,8.11,-1.02,5.58,6.84,5.53,NaN,NaN,8.98,-

In [5]:
normalizated_train_set = no_normalized_train_set.copy()   
normalizated_train_set = normalization(normalizated_train_set)
normalizated_train_set

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,2,100,1.618125,-2.751875,NaN,NaN,-4.841875,-12.121875,-3.191875,-7.801875,6.418125,6.758125,4.288125,6.178125,1.958125,4.968125,NaN,NaN,2.198125,-3.141875,NaN,-3.671875,-1.591875,NaN,5.888125,6.708125,-2.411875,NaN,2.248125,-1.591875,-2.851875,4.528125,4.038125,-3.381875,4.678125,NaN,-4.261875,-1.731875,4.628125,NaN,-3.331875,5.448125,5.358125,-6.291875,NaN,6.518125,-6.781875,-5.661875,-7.361875,NaN,-3.921875,1.178125,0.698125,NaN,-1.491875,-3.771875,NaN,-5.661875,6.178125,-4.601875,NaN,6.568125,-9.161875,-5.811875,-11.491875,2.008125,1.618125,-6.291875,6.278125,-1.341875,-1.681875,5.058125,-7.461875,0.308125,5.838125,5.308125,4.868125,3.748125,5.258125,6.518125,6.178125,5.738125,NaN,2.388125,2.388125,NaN,6.518125,2.048125,-2.511875,1.228125,2.098125,NaN,NaN,-7.411875,-2.751875,NaN,NaN,-4.601875,0.598125,-2.121875,-6.781875,-1.391875
1,6,100,-1.218375,1.411625,5.391625,-3.548375,-2.138375,0.631625,-3.738375,NaN,-1.698375,3.151625,NaN,-0.778375,-0.048375,-3.638375,5.441625,-3.978375,1.261625,2.771625,2.671625,-1.168375,1.941625,4.371625,NaN,-2.818375,2.721625,NaN,-0.288375,NaN,4.271625,-1.988375,1.021625,NaN,-1.648375,-0.538375,3.401625,4.461625,-1.258375,-3.108375,NaN,-2.088375,-3.878375,-2.958375,6.021625,3.061625,-1.408375,NaN,NaN,NaN,-2.868375,4.321625,-1.358375,-2.528375,NaN,1.501625,1.751625,5.481625,-4.318375,-4.608375,NaN,3.831625,NaN,2.091625,0.341625,-4.468375,-2.958375,-4.858375,-4.468375,-4.028375,3.011625,-2.038375,-4.708375,-0.238375,-0.048375,0.531625,-2.328375,-1.548375,0.631625,-1.988375,NaN,NaN,-1.258375,2.091625,-2.668375,-1.168375,4.661625,NaN,1.021625,3.011625,-0.728375,NaN,NaN,-1.938375,NaN,1.991625,2.771625,1.601625,5.001625,NaN,-0.098375,1.501625
2,8,100,9.327250,5.647250,11.657250,-3.722750,-5.672750,NaN,11.757250,NaN,-2.702750,-1.932750,10.687250,-5.372750,-4.452750,NaN,2.777250,NaN,-4.602750,-4.692750,3.507250,NaN,NaN,NaN,3.607250,-5.762750,6.277250,NaN,5.937250,-0.962750,NaN,-1.682750,2.047250,NaN,8.317250,NaN,-2.952750,NaN,-4.652750,5.497250,-3.582750,NaN,-3.972750,-1.932750,NaN,3.017250,1.227250,-1.932750,-6.442750,11.417250,2.387250,9.917250,-2.022750,-4.892750,6.657250,-6.542750,0.347250,3.797250,-3.192750,NaN,-3.722750,NaN,-0.712750,-3.632750,-0.422750,-6.732750,-0.132750,-4.842750,-0.762750,-1.732750,-7.322750,0.397250,-7.412750,-1.302750,2.977250,NaN,-7.362750,4.287250,-1.682750,NaN,-6.642750,-6.392750,6.027250,-2.072750,NaN,-3.482750,-7.412750,2.147250,-1.642750,2.387250,2.247250,-5.472750,9.717250,1.367250,2.387250,-3.192750,-0.672750,-0.862750,4.627250,2.437250,NaN,2.487250
3,9,100,-2.978250,NaN,-8.608250,-6.078250,-7.928250,NaN,-4.478250,-8.118250,NaN,NaN,-2.098250,0.521750,-4.038250,NaN,-7.928250,-6.178250,-7.928250,-2.098250,NaN,NaN,4.791750,NaN,0.521750,2.021750,4.601750,3.721750,5.661750,3.091750,1.781750,0.521750,7.171750,1.341750,NaN,5.471750,4.161750,1.781750,-2.298250,NaN,NaN,-3.608250,-0.598250,2.221750,-4.528250,-6.178250,2.221750,5.911750,2.651750,0.521750,1.341750,0.521750,-6.178250,-2.728250,NaN,2.651750,4.161750,0.521750,0.521750,-4.478250,-6.078250,-4.478250,NaN,6.341750,2.221750,-5.688250,5.661750,5.661750,3.431750,2.221750,7.171750,2.221750,-3.948250,4.981750,NaN,

# Valutazione MAE

In [6]:
def mae(final, test_set): 
    t1 = 0 
    for i, j in zip(final, test_set):
        joke = ''
        for k in i.keys():
            if k.startswith('joke'):
                joke = k
        if joke in i and joke in j:
            t1+=abs(i[joke] - j[joke])
    return t1 / len(final)

# Valutazione RMSE

In [7]:
def rmse(final, test_set): 
    t1 = 0 
    for i, j in zip(final, test_set):
        joke = ''
        for k in i.keys():
            if k.startswith('joke'):
                joke = k
        if joke in i and joke in j:
            t1+=(i[joke] - j[joke]) * (i[joke] - j[joke])
    return np.sqrt(t1 / len(final))

# Raccomandazione UBCF

In [8]:
final = []

# per ogni utente nel training set
for n in list(normalizated_train_set["user_id"]):
    lst = recommendation_ubcf(n, normalizated_train_set, no_normalized_train_set, True)
    final = final + lst

# Raccomandazione IBCF

In [9]:
# si costruisce la matrice di similarità, ovvero una lista di coppie di item, per 
# ogni coppia si specifica la similarità (ad -1 a 1)
all_jokes = (list(no_normalized_train_set.columns))[2:]
jokes_pairs = list(combinations(all_jokes, 2))
similarity_matrix = []

for i_1, i_2 in jokes_pairs:
    app = normalizated_train_set[[i_1, i_2]].dropna()
    similarity_matrix.append([i_1, i_2, cosine_similarity(app[i_1], app[i_2])])
    
df = pd.DataFrame(similarity_matrix, columns = ['elem_1', 'elem_2', 'similarity'])
df = df[df['similarity'] > 0.3]
df

,elem_1,elem_2,similarity
1,joke_1,joke_3,0.392730
2,joke_1,joke_4,0.435194
3,joke_1,joke_5,0.432674
5,joke_1,joke_7,0.340823
9,joke_1,joke_11,0.345801
...,...,...,...
4907,joke_91,joke_94,0.474522
4908,joke_91,joke_95,0.485318
4917,joke_92,joke_96,0.477840
4938,joke_95,joke_99,0.527282


In [10]:
final_2 = []
for n in list(complete_ratings["user_id"]):
    lst = recommendation_ibcf(n, no_normalized_train_set, df, True)
    final_2 = final_2 + lst

In [11]:
# final

In [12]:
test_set.sort(key=lambda x: (x["user_id"],int(list(x.keys())[1][5:])), reverse=False)
#test_set

# Confronto: UBCF vs. ITCF

In [13]:
print("MAE UBCF: " + str(mae(final, test_set)))
print("RMSE UBCF: " + str(rmse(final, test_set)))

print("MAE IBCF: " + str(mae(final_2, test_set)))
print("RMSE IBCF: " + str(rmse(final_2, test_set)))

MAE UBCF: 4.329469375176718
RMSE UBCF: 5.453721216366851
MAE IBCF: 3.8872036392614264
RMSE IBCF: 5.036507794041863
